In [17]:
###加载数据

import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# 设置绘图风格和中文显示
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False  

# --- 加载我们上一阶段准备好的最终数据 ---
try:
    # 请确保文件名与您保存的文件一致
    data_path = r"E:\A智网\电量预测数据\lightgbm模型\133个行业电量气象对齐数据.csv"
    df_panel = pd.read_csv(data_path, parse_dates=['date'], encoding='gbk')
    print("数据加载成功！")
    print(df_panel.head())

except FileNotFoundError:
    print(f"错误：文件未找到 {data_path}，请检查路径。")
    df_panel = pd.DataFrame()
except UnicodeDecodeError:
    # 如果'gbk'也不行，说明文件可能被意外地用GBK编码覆盖了
    print("使用 'gbk' 编码失败，这很奇怪。正在尝试 'utf-8' 作为备用方案...")
    try:
        df_panel = pd.read_csv(data_path, parse_dates=['date'], encoding='utf-8')
        print("数据加载成功！")
        print(df_panel.head())
    except Exception as e:
        print(f"尝试所有编码后依然加载失败: {e}")
        df_panel = pd.DataFrame()
except Exception as e:
    print(f"加载数据时发生未知错误: {e}")
    df_panel = pd.DataFrame()

数据加载成功！
        行业名称       date    load_MWh  temp_mean  temp_max  temp_min  temp_std  \
0      客户用电量 2023-01-01  56822.2209      6.883     8.776     5.495      1.09   
1  A、全行业用电合计 2023-01-01  39712.4177      6.883     8.776     5.495      1.09   
2       第一产业 2023-01-01    811.9297      6.883     8.776     5.495      1.09   
3       第二产业 2023-01-01  25598.1050      6.883     8.776     5.495      1.09   
4       第三产业 2023-01-01  13302.3830      6.883     8.776     5.495      1.09   

   temp_range  
0       3.281  
1       3.281  
2       3.281  
3       3.281  
4       3.281  


In [18]:
# ==============================================================================
# 模块二(V4 - 最终完美版)：在工作日数据上进行滚动窗口异常检测
# 目的：精准识别“噪音”异常，同时完全保护“节假日/周末效应”这一宝贵信号。
# ==============================================================================
from chinese_calendar import is_workday

def flag_outliers_on_workdays(df_panel, industry_col='行业名称', value_col='load_MWh', date_col='date', window_size=31, factor=2.5):
    """
    只在工作日数据上，使用滚动窗口IQR方法，进行异常检测并创建标记特征。
    """
    print("\n" + "="*50)
    print("      开始执行：工作日滚动窗口异常检测")
    print("="*50)
    
    df_flagged = df_panel.copy()
    # 1. 【核心新增】首先，识别出所有的工作日
    workday_mask = df_flagged[date_col].apply(is_workday)
    
    # 创建一个新列来存放异常标记，默认所有都不是异常
    df_flagged['is_outlier'] = 0
    
    industries = df_flagged[industry_col].unique()
    total_outliers_found = 0

    try:
        from tqdm import tqdm
        print("正在逐一分析各行业的工作日数据...")
        iterator = tqdm(industries)
    except ImportError:
        iterator = industries

    for industry in iterator:
        # 2. 【核心新增】只筛选出当前行业的“工作日”数据进行分析
        industry_workday_mask = (df_flagged[industry_col] == industry) & workday_mask
        series = df_flagged.loc[industry_workday_mask, value_col]
        
        if len(series) < 20: continue # 如果工作日数据太少，则跳过
            
        # 3. 在工作日数据上计算滚动的分位数和IQR
        rolling_q1 = series.rolling(window=window_size, center=True, min_periods=window_size//2).quantile(0.25)
        rolling_q3 = series.rolling(window=window_size, center=True, min_periods=window_size//2).quantile(0.75)
        rolling_iqr = rolling_q3 - rolling_q1
        
        # 4. 定义滚动的异常边界 (进一步放宽factor到2.5，更严格地定义异常)
        lower_bound = rolling_q1 - (rolling_iqr * factor)
        upper_bound = rolling_q3 + (rolling_iqr * factor)
        
        # 5. 找出超出局部边界的异常点
        outlier_mask = (series < lower_bound) | (series > upper_bound)
        outlier_indices = series[outlier_mask].index
        
        # 6. 标记异常
        df_flagged.loc[outlier_indices, 'is_outlier'] = 1
        total_outliers_found += outlier_mask.sum()

    print(f"\n处理完成！总共在【工作日】数据中标记了 {total_outliers_found} 个统计异常点。")
    print("="*50 + "\n")
    return df_flagged

# --- 调用最终版标记函数 ---
if 'df_panel' in locals() and not df_panel.empty:
    df_flagged = flag_outliers_on_workdays(df_panel)
    df_panel = df_flagged
    print("原始变量 'df_panel' 已被添加了'is_outlier'特征的数据覆盖。")
    
    if df_panel[df_panel['is_outlier'] == 1].shape[0] > 0:
        print("\n新特征预览 (部分被标记为异常的工作日):")
        print(df_panel[df_panel['is_outlier'] == 1].head())
    else:
        print("\n在新的检测方法下，工作日数据中未发现任何需要标记的异常点。")
else:
    print("错误：原始数据 'df_panel' 未加载，无法进行异常处理。")


      开始执行：工作日滚动窗口异常检测
正在逐一分析各行业的工作日数据...


100%|██████████| 134/134 [00:00<00:00, 156.23it/s]


处理完成！总共在【工作日】数据中标记了 563 个统计异常点。

原始变量 'df_panel' 已被添加了'is_outlier'特征的数据覆盖。

新特征预览 (部分被标记为异常的工作日):
              行业名称       date   load_MWh  temp_mean  temp_max  temp_min  \
388         其中：租赁业 2023-01-03   273.2038      5.711    11.349     1.355   
491     抽水蓄能抽水耗用电量 2023-01-04     0.0211      6.257    11.328     0.901   
597         其中：铝冶炼 2023-01-05     0.3440      8.434    11.239     6.407   
2124      六、批发和零售业 2023-01-16  3391.8149      0.083     2.851    -3.726   
2458      其中：中成药生产 2023-01-19    30.0564      5.884    13.018     0.139   

      temp_std  temp_range  is_outlier  
388      3.217       9.994           1  
491      3.881      10.427           1  
597      1.528       4.832           1  
2124     2.458       6.577           1  
2458     4.153      12.879           1  


In [19]:
# --- 步骤：模型分层 (修正版 - 在特征工程前进行) ---

# 【重要】确保这个代码块在"异常检测与标记"之后，但在"特征工程"之前

if 'df_panel' in locals() and not df_panel.empty:
    print("\n" + "="*50)
    print("      开始执行：模型分层 (修正版)")
    print("="*50)

    # 1. 【核心修正】在进行特征工程之前，在最完整的数据'df_panel'上计算日均用电量
    print("正在基于最完整的数据计算各行业日均用电量...")
    industry_avg_load = df_panel.groupby('行业名称')['load_MWh'].mean().to_frame('avg_load')

    # --- 您的描述性统计代码可以放在这里 ---
    print("\n--- 行业日均用电量描述性统计 ---")
    print(industry_avg_load['avg_load'].describe().to_string(float_format='{:,.2f}'.format))
    
    # 2. 根据我们分析得出的阈值来定义行业层级
    def assign_tier(avg_load):
        if avg_load >= 1000:
            return '大行业'
        elif avg_load < 100:
            return '小行业'
        else:
            return '中行业'

    industry_avg_load['tier'] = industry_avg_load['avg_load'].apply(assign_tier)

    print("\n各行业层级划分预览：")
    print(industry_avg_load.head())
    print("\n各层级行业数量统计:")
    print(industry_avg_load['tier'].value_counts())

    # 3. 【核心修正】将层级信息合并回主数据集'df_panel'
    df_panel = df_panel.merge(industry_avg_load[['tier']], on='行业名称', how='left')

    print("\n已将层级信息'tier'合并至主DataFrame 'df_panel'。")
    print("现在可以继续进行特征工程了。")

else:
    print("错误：DataFrame 'df_panel' 不存在或为空，无法进行模型分层。")




      开始执行：模型分层 (修正版)
正在基于最完整的数据计算各行业日均用电量...

--- 行业日均用电量描述性统计 ---
count      134.00
mean     2,890.10
std      9,534.80
min          0.00
25%         78.83
50%        314.92
75%      1,003.86
max     65,324.45

各行业层级划分预览：
           avg_load tier
行业名称                    
  其中：排灌  339.523322  中行业
1.农业     517.533557  中行业
2.林业       8.736954  小行业
3.畜牧业    411.720397  中行业
4.渔业     140.766090  中行业

各层级行业数量统计:
tier
中行业    63
小行业    37
大行业    34
Name: count, dtype: int64

已将层级信息'tier'合并至主DataFrame 'df_panel'。
现在可以继续进行特征工程了。


In [24]:
#导入我们需要的库
#特征工程


from chinese_calendar import is_workday, is_holiday

def create_panel_features_with_holidays(df):
    """
    为面板数据创建特征 (包含节假日特征)
    """
    df = df.copy()
    
    # 确保数据按行业和日期排序
    df = df.sort_values(by=['行业名称', 'date']).reset_index(drop=True)
    
    # --- 1. 时间特征 ---
    df['month'] = df['date'].dt.month
    df['dayofweek'] = df['date'].dt.dayofweek
    df['dayofyear'] = df['date'].dt.dayofyear
    df['weekofyear'] = df['date'].dt.isocalendar().week.astype(int)
    
    # --- 【核心新增】创建节假日特征 ---
    # is_holiday() 返回 True / False
    # is_workday() 对于调休的工作日也返回 True
    
    # a) 是否为法定节假日 (如国庆节当天)
    df['is_holiday'] = df['date'].apply(is_holiday).astype(int)
    
    # b) 是否为周末 (正常的周六日)
    df['is_weekend_norm'] = (df['date'].dt.dayofweek >= 5).astype(int)
    
    # c) 是否为调休的工作日 (本应是周末但需要上班)
    # is_workday返回True, 且 dayofweek >= 5 (是周末)
    df['is_adj_workday'] = df.apply(lambda row: 1 if is_workday(row['date']) and row['dayofweek'] >= 5 else 0, axis=1)

    # d) 最终的“休息日”特征：是节假日，或者是“正常的”周末（排除了调休）
    df['is_offday'] = df.apply(lambda row: 1 if row['is_holiday'] == 1 or (row['is_weekend_norm'] == 1 and row['is_adj_workday'] == 0) else 0, axis=1)

    print("节假日特征创建预览：")
    # 找一个包含节假日的日期范围来检查
    holiday_check_df = df[df['date'].between('2024-04-28', '2024-05-06')].drop_duplicates(subset=['date'])
    print(holiday_check_df[['date', 'dayofweek', 'is_holiday', 'is_weekend_norm', 'is_adj_workday', 'is_offday']])
    
    # --- 2. 滞后特征 (Lag Features) ---
    lags = [1, 2, 7, 14]
    for lag in lags:
        df[f'load_lag_{lag}'] = df.groupby('行业名称')['load_MWh'].shift(lag)
        
    # --- 3. 窗口特征 (Rolling Window Features) ---
    df['rolling_mean_7'] = df.groupby('行业名称')['load_MWh'].shift(1).rolling(window=7, min_periods=1).mean()
    df['rolling_std_7'] = df.groupby('行业名称')['load_MWh'].shift(1).rolling(window=7, min_periods=1).std()
    
    # --- 5. 【核心新增】高级差异/比率特征 ---
    print("正在创建高级差异/比率特征...")
    
    # a) 昨天的电量与上周同一天电量的差异
    #    这个特征能直接告诉模型“周同比”的变化趋势
    df['load_diff_1_7'] = df['load_lag_1'] - df['load_lag_7']
    
    # b) 昨天的电量与过去7天平均值的差异比率
    #    这个特征能量化“昨天”是否是一个反常的高点或低点
    #    + 1e-6 是为了防止除以0的错误
    df['load_ratio_1_roll7'] = (df['load_lag_1'] / (df['rolling_mean_7'] + 1e-6)) -1

    print("高级特征创建完成！")


    # --- 6. 行业本身作为特征 ---
    df['行业名称'] = df['行业名称'].astype('category')
    
    # 【清理】删除我们用于计算的中间特征
    df = df.drop(['is_weekend_norm', 'is_adj_workday'], axis=1)
    
    return df

# --- 调用新的特征工程函数 ---
if not df_panel.empty:
    print("\n开始进行特征工程 (包含节假日特征)...")
    df_featured = create_panel_features_with_holidays(df_panel)

    # 删除因创建滞后特征而产生的空值行
    df_featured = df_featured.dropna()
    
    print("\n特征工程完成！数据预览：")
    print(df_featured.head())
    print("\n特征列数据类型：")
    df_featured.info()


开始进行特征工程 (包含节假日特征)...
节假日特征创建预览：
          date  dayofweek  is_holiday  is_weekend_norm  is_adj_workday  \
483 2024-04-28          6           0                1               1   
484 2024-04-29          0           0                0               0   
485 2024-04-30          1           0                0               0   
486 2024-05-01          2           1                0               0   
487 2024-05-02          3           1                0               0   
488 2024-05-03          4           1                0               0   
489 2024-05-04          5           1                1               0   
490 2024-05-05          6           1                1               0   
491 2024-05-06          0           0                0               0   

     is_offday  
483          0  
484          0  
485          0  
486          1  
487          1  
488          1  
489          1  
490          1  
491          0  
正在创建高级差异/比率特征...
高级特征创建完成！

特征工程完成！数据预览：
           行业名

In [25]:
# ==============================================================================
# 模块五 (最终修正版): 数据集拆分
# 输入: df_featured
# 输出: data_tiers (一个包含三个DataFrame的字典)
# ==============================================================================

# 创建一个空字典，用于存放拆分后的数据集
data_tiers = {}

if 'df_featured' in locals() and 'tier' in df_featured.columns:
    print("\n--- 正在将特征化后的数据集按层级拆分 ---")
    
    # 直接在字典中创建DataFrame
    data_tiers['大行业'] = df_featured[df_featured['tier'] == '大行业'].copy()
    data_tiers['中行业'] = df_featured[df_featured['tier'] == '中行业'].copy()
    data_tiers['小行业'] = df_featured[df_featured['tier'] == '小行业'].copy()

    print(f"数据集拆分完成，并已存入 'data_tiers' 字典。")
    print(f"  大行业数量: {data_tiers['大行业']['行业名称'].nunique()}, 数据点: {len(data_tiers['大行业'])}")
    print(f"  中行业数量: {data_tiers['中行业']['行业名称'].nunique()}, 数据点: {len(data_tiers['中行业'])}")
    print(f"  小行业数量: {data_tiers['小行业']['行业名称'].nunique()}, 数据点: {len(data_tiers['小行业'])}")
else:
    print("错误：特征化后的数据 'df_featured' 不存在或缺少'tier'列，无法拆分。")


--- 正在将特征化后的数据集按层级拆分 ---
数据集拆分完成，并已存入 'data_tiers' 字典。
  大行业数量: 34, 数据点: 30532
  中行业数量: 63, 数据点: 56574
  小行业数量: 37, 数据点: 33226


In [ ]:
import optuna
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import json

# --- 模块：为三个分层模型分别进行超参数调优 (在您的版本上修正) ---

# 【核心修正】
# 我们不再依赖'locals()'这种可能不稳定的检查方式。
# 我们直接尝试创建字典，并检查每个DataFrame是否真实存在且不为空。

data_tiers = {}
print("正在准备用于调优的数据集...")

# 检查'df_large'是否存在且有数据
if 'df_large' in locals() and not df_large.empty:
    data_tiers['大行业'] = df_large
    print("  - '大行业' 数据集已成功加载。")
else:
    print("  - [警告] 未找到或'df_large'为空。")

# 检查'df_medium'是否存在且有数据
if 'df_medium' in locals() and not df_medium.empty:
    data_tiers['中行业'] = df_medium
    print("  - '中行业' 数据集已成功加载。")
else:
    print("  - [警告] 未找到或'df_medium'为空。")

# 检查'df_small'是否存在且有数据
if 'df_small' in locals() and not df_small.empty:
    data_tiers['小行业'] = df_small
    print("  - '小行业' 数据集已成功加载。")
else:
    print("  - [警告] 未找到或'df_small'为空。")


# 如果data_tiers字典最终不为空，才继续执行
if data_tiers:
    print("\n数据准备就绪，开始调优流程。")
    
    # 创建一个空字典，用于自动存储每个模型的最优参数
    best_params_per_tier = {}

    print("\n" + "="*50)
    print("      开始为三个层级的模型进行自动化超参数调优")
    print("="*50)

    for tier_name, df_tier in data_tiers.items():
        if df_tier.empty: continue
        print(f"\n--- 正在为【{tier_name}】模型寻找最优参数 ---")
        
        # 1. 数据划分
        val_date_start = df_tier['date'].max() - pd.DateOffset(days=60)
        val_date_end = df_tier['date'].max() - pd.DateOffset(days=31)
        
        train = df_tier[df_tier['date'] < val_date_start].copy()
        val = df_tier[(df_tier['date'] >= val_date_start) & (df_tier['date'] <= val_date_end)].copy()

        FEATURES = [col for col in df_tier.columns if col not in ['date', 'load_MWh', 'tier']]
        TARGET = 'load_MWh'
        
        X_train, y_train = train[FEATURES], train[TARGET]
        X_val, y_val = val[FEATURES], val[TARGET]

        # 2. 定义Optuna的目标函数
        def objective(trial):
            params = {
                'objective': 'regression_l1', 'metric': 'l1', 'n_estimators': 2000,
                'random_state': 42, 'n_jobs': -1, 'verbosity': -1,
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
            }
            
            model = lgb.LGBMRegressor(**params)
            model.fit(X_train, y_train,
                      eval_set=[(X_val, y_val)],
                      eval_metric='mae',
                      callbacks=[lgb.early_stopping(100, verbose=False)],
                      categorical_feature=['行业名称'])
            
            preds = model.predict(X_val)
            mae = mean_absolute_error(y_val, preds)
            return mae

        # 3. 创建并运行Optuna研究
        study = optuna.create_study(direction='minimize') 
        study.optimize(objective, n_trials=50, show_progress_bar=True)
        
        # 4. 自动将找到的最佳参数存入我们的字典中
        best_params_per_tier[tier_name] = study.best_params
        
        print(f"【{tier_name}】模型调优完成！")
        print(f"  最佳验证集 MAE: {study.best_value:.2f}")
        print(f"  找到的最佳参数已自动存储。")

    print("\n" + "="*50)
    print("      所有模型的超参数调优均已完成！")
    print("="*50)
    print("最终找到的各层级最优参数为：")
    print(json.dumps(best_params_per_tier, indent=4))

else:
    print("\n错误：未能成功加载任何分层数据集 (df_large, df_medium, df_small)。请确保“数据集拆分”代码块已成功运行。")

In [ ]:
# --- 模块二：使用自动存储的最优参数进行最终训练与评估 (功能增强版) ---

#MAE=91.1
best_params_per_tier = {
    '大行业': {'learning_rate': 0.034002868380916096, 'num_leaves': 48, 'max_depth': 10, 'min_child_samples': 36, 'feature_fraction': 0.8655495783244953, 'bagging_fraction': 0.7136846508742616, 'bagging_freq': 7, 'lambda_l1': 0.0032244564626965682, 'lambda_l2': 4.603059648930217e-05},
    '中行业': {'learning_rate': 0.06288131356160809, 'num_leaves': 179, 'max_depth': 9, 'min_child_samples': 82, 'feature_fraction': 0.8859465064109061, 'bagging_fraction': 0.645748842980119, 'bagging_freq': 6, 'lambda_l1': 6.785845799108288e-06, 'lambda_l2': 9.943664916758198},
    '小行业': {'learning_rate': 0.02804580180260286, 'num_leaves': 228, 'max_depth': 9, 'min_child_samples': 68, 'feature_fraction': 0.9979956267652822, 'bagging_fraction': 0.9188841309431092, 'bagging_freq': 5, 'lambda_l1': 0.15471069640191787, 'lambda_l2': 0.4707028322072485}
}


if 'best_params_per_tier' in locals() and best_params_per_tier:
    
    final_models = {}
    final_predictions = []

    print("\n" + "="*50)
    print("      使用【自动读取】的最优参数进行最终训练与评估")
    print("="*50)

    # ... (前面的循环训练部分保持不变) ...
    for tier_name, df_tier in data_tiers.items():
        if df_tier.empty: continue
        print(f"\n--- 正在处理【{tier_name}】模型 (最终训练) ---")
        
        split_date = df_tier['date'].max() - pd.DateOffset(days=30)
        train = df_tier[df_tier['date'] <= split_date].copy()
        test = df_tier[df_tier['date'] > split_date].copy()
        
        FEATURES = [col for col in df_tier.columns if col not in ['date', 'load_MWh', 'tier']]
        TARGET = 'load_MWh'
        
        X_train, y_train = train[FEATURES], train[TARGET]
        X_test, y_test = test[FEATURES], test[TARGET]

        print(f"正在为【{tier_name}】加载最优参数...")
        params = best_params_per_tier[tier_name]
        params.update({
            'objective': 'regression_l1', 'random_state': 42,
            'n_jobs': -1, 'n_estimators': 5000
        })

        model = lgb.LGBMRegressor(**params)
        
        print(f"开始使用最优参数训练【{tier_name}】模型...")
        model.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)], eval_metric='mae',
                  callbacks=[lgb.early_stopping(200, verbose=False)],
                  categorical_feature=['行业名称'])
                  
        final_models[tier_name] = model
        print(f"【{tier_name}】模型训练完成。最佳迭代次数: {model.best_iteration_}")
        
        test['prediction'] = model.predict(X_test)
        final_predictions.append(test)
        
    # --- 【核心修正】恢复并增强最终的评估报告 ---
    if final_predictions:
        # 1. 合并所有层级的预测结果
        final_test_df_tuned = pd.concat(final_predictions)
        
        # 2. 计算【总体】性能指标
        overall_mae = mean_absolute_error(final_test_df_tuned[TARGET], final_test_df_tuned['prediction'])
        overall_rmse = np.sqrt(mean_squared_error(final_test_df_tuned[TARGET], final_test_df_tuned['prediction']))
        
        # 计算总体MAPE时，需要排除真实值为0的情况
        non_zero_mask_overall = final_test_df_tuned[TARGET] != 0
        overall_mape = mean_absolute_percentage_error(
            final_test_df_tuned[TARGET][non_zero_mask_overall], 
            final_test_df_tuned['prediction'][non_zero_mask_overall]
        )

        print("\n" + "="*50)
        print("      【最终版】调优后模型【总体性能评估】")
        print("="*50)
        print(f"  整体平均绝对误差 (MAE): {overall_mae:,.2f}")
        print(f"  整体均方根误差 (RMSE): {overall_rmse:,.2f}")
        print(f"  整体平均绝对百分比误差 (MAPE): {overall_mape:.2%}")

        # 3. 计算并展示【重点行业】的详细性能报告
        #    (复用我们之前最强大的评估模块)
        key_industries = [
            "客户用电量", "第二产业", "B、城乡居民生活用电合计",
            "14.化学原料和化学制品制造业", "15.医药制造业", "18.非金属矿物制品业",
            "19.黑色金属冶炼和压延加工业", "20.有色金属冶炼和压延加工业",
            "24.汽车制造业", "26.电气机械和器材制造业", "九、房地产业", "其中：地质勘查"
        ]
        
        print("\n" + "="*60)
        print("      重点行业性能评估报告 (最终版)")
        print("="*60)
    
        # (此处省略了和之前完全一样的、计算重点行业指标并打印表格的代码)
        # (确保您脚本中保留了那个详细的评估模块)
        # 假设 df_key_metrics 是计算出的结果
        # print(df_key_metrics.to_string(...))

        # 4. 可视化【重点行业】的预测结果
        print("\n--- 重点行业预测结果可视化 ---")
        # (此处省略了和之前完全一样的、为重点行业循环绘图的代码)
        # for industry in key_industries:
        #     # ... 绘图代码 ...
        #     plt.show()

    else:
        print("未能成功训练任何模型。")
else:
    print("\n错误：未能找到最优参数字典 'best_params_per_tier'。")


      使用【自动读取】的最优参数进行最终训练与评估

--- 正在处理【大行业】模型 (最终训练) ---
正在为【大行业】加载最优参数...
开始使用最优参数训练【大行业】模型...
[LightGBM] [Warning] feature_fraction is set=0.8655495783244953, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8655495783244953
[LightGBM] [Warning] lambda_l1 is set=0.0032244564626965682, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0032244564626965682
[LightGBM] [Warning] lambda_l2 is set=4.603059648930217e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.603059648930217e-05
[LightGBM] [Warning] bagging_fraction is set=0.7136846508742616, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7136846508742616
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8655495783244953, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8655495783244953
[LightGBM] [Warning] lambda_l1 is set=0.0032244564626965682, 

In [ ]:
# --- 步骤：重点行业可视化评估 (针对分层模型结果) ---

# 确认包含所有预测结果的DataFrame 'final_test_df_tuned' 已经创建
if 'final_test_df_tuned' in locals() and not final_test_df_tuned.empty:

    # --- 1. 定义您重点关注的行业列表 ---
    key_industries = [
        "客户用电量",
        "    第二产业",
        "    第三产业",
        "B、城乡居民生活用电合计",
        "14.化学原料和化学制品制造业",
        "15.医药制造业",
        "18.非金属矿物制品业",
        "19.黑色金属冶炼和压延加工业",
        "20.有色金属冶炼和压延加工业",
        "24.汽车制造业",
        "26.电气机械和器材制造业",
        "九、房地产业",
        "  其中：地质勘查"
        # 您可以按需添加或修改这个列表
    ]
    
    # 检查您关注的行业是否都存在于测试集中
    missing_industries = [ind for ind in key_industries if ind not in final_test_df_tuned['行业名称'].unique()]
    if missing_industries:
        print(f"\n警告：以下您关注的行业在测试集中不存在，将被忽略: {missing_industries}")
        key_industries = [ind for ind in key_industries if ind in final_test_df_tuned['行业名称'].unique()]

    # --- 2. 为重点行业计算详细的评估指标 ---
    print("\n" + "="*60)
    print("      重点行业性能评估报告 (分层模型版)")
    print("="*60)
    
    metrics_list = []
    for industry in key_industries:
        subset = final_test_df_tuned[final_test_df_tuned['行业名称'] == industry]
        if subset.empty: continue
            
        true_values = subset[TARGET]
        pred_values = subset['prediction']
        
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        
        non_zero_mask = true_values != 0
        mape = np.nan if not np.any(non_zero_mask) else mean_absolute_percentage_error(true_values[non_zero_mask], pred_values[non_zero_mask])
        
        avg_load = true_values.mean()

        metrics_list.append({
            '行业名称': industry,
            '所属模型层级': subset['tier'].iloc[0], # 【新】获取该行业所属的层级
            '平均绝对误差 (MAE)': mae,
            '均方根误差 (RMSE)': rmse,
            '平均绝对百分比误差 (MAPE)': mape,
            '日均用电量': avg_load,
            'MAE占日均比': mae / avg_load if avg_load != 0 else np.nan
        })

    df_key_metrics = pd.DataFrame(metrics_list).set_index('行业名称')
    
    # 打印格式化的表格
    pd.options.display.float_format = '{:,.2f}'.format
    print(df_key_metrics.to_string(formatters={
        '平均绝对百分比误差 (MAPE)': '{:,.2%}'.format,
        'MAE占日均比': '{:,.2%}'.format
    }))
    
    # --- 3. 为每个重点行业绘制高质量的预测对比图 ---
    print("\n--- 重点行业预测结果可视化 ---")
    
    for industry in key_industries:
        subset = final_test_df_tuned[final_test_df_tuned['行业名称'] == industry]
        if subset.empty: continue
            
        fig, ax = plt.subplots(figsize=(20, 6))
        
        ax.plot(subset['date'], subset[TARGET], label='真实值', marker='.', linestyle='-', linewidth=2)
        ax.plot(subset['date'], subset['prediction'], label='预测值', marker='x', linestyle='--', markersize=8)
        
        ax.set_title(f"行业 '{industry}' (由【{subset['tier'].iloc[0]}】模型预测) 的预测结果", fontsize=18, pad=20)
        ax.set_xlabel('日期', fontsize=12)
        ax.set_ylabel('电量 (MWh)', fontsize=12)
        
        ax.legend(fontsize=12, loc='upper right')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        
        y_min = min(subset[TARGET].min(), subset['prediction'].min())
        y_max = max(subset[TARGET].max(), subset['prediction'].max())
        ax.set_ylim(y_min * 0.95, y_max * 1.05)
        
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.show()

else:
    print("错误：未能找到名为 'final_test_df_tuned' 的DataFrame。请确保您已经成功运行了模型分层的训练代码块。")

In [ ]:
def predict_future(historical_data, days_to_predict, models, industry_tiers):
    """
    使用分层模型，对未来N天的所有行业电量进行递归预测。

    参数:
    - historical_data: DataFrame, 包含了到今天为止的所有历史数据 
                       (至少需要包含 'date', '行业名称', 'load_MWh', 以及所有天气特征列)。
    - days_to_predict: int, 您想要预测的未来天数 (例如: 8)。
    - models: dict, 存放了我们训练好的三个分层模型 {'大行业': model, ...}。
    - industry_tiers: DataFrame, 包含了每个行业的层级信息。

    返回:
    - DataFrame, 包含了未来N天所有行业的预测结果。
    """
    print(f"\n--- 开始预测未来 {days_to_predict} 天的电量 ---")
    
    # 1. 创建未来的日期范围
    last_known_date = historical_data['date'].max()
    future_dates = pd.date_range(start=last_known_date + pd.DateOffset(days=1), periods=days_to_predict, freq='D')
    
    # 2. 准备一个DataFrame来存放预测结果
    #    我们需要为每个行业都创建未来的行
    unique_industries = historical_data['行业名称'].unique()
    future_df = pd.DataFrame({
        'date': np.repeat(future_dates, len(unique_industries)),
        '行业名称': np.tile(unique_industries, days_to_predict)
    })

    # 3. 【关键】将历史数据和未来框架合并，方便进行递归特征计算
    #    我们只保留能用于计算滞后特征的最后一部分历史数据（例如，过去30天）
    history_for_features = historical_data[historical_data['date'] >= last_known_date - pd.DateOffset(days=30)].copy()
    full_df = pd.concat([history_for_features, future_df], ignore_index=True)

    # 4. 【占位符】为未来日期填充天气数据
    #    !!! 重要 !!!
    #    在这里，我们暂时使用最后一个已知日期的天气来“前向填充”未来的天气。
    #    当您收集到真实的未来天气预报后，需要在这里将其合并进来。
    weather_cols = [col for col in historical_data.columns if 'temp_' in col]
    full_df[weather_cols] = full_df[weather_cols].fillna(method='ffill')

    # 5. 【核心】开始逐日递归预测
    for current_date in future_dates:
        print(f"正在预测 {current_date.date()}...")
        
        # a) 为当前需要预测的日期创建完整的特征
        #    注意: 我们必须传入整个 full_df，以便特征工程函数能计算滞后值
        df_with_features = create_panel_features_with_holidays(full_df)
        
        # b) 筛选出我们正要预测的这一天的数据
        predict_day_data = df_with_features[df_with_features['date'] == current_date].copy()
        
        # c) 加入行业层级信息，以便我们知道该用哪个模型
        predict_day_data = predict_day_data.merge(industry_tiers[['tier']], on='行业名称', how='left')
        
        # d) 按层级进行预测
        for tier_name, model in models.items():
            # 找到属于这个层级的行业
            tier_industries_mask = (predict_day_data['tier'] == tier_name)
            
            if tier_industries_mask.any():
                # 准备特征矩阵
                X_pred = predict_day_data.loc[tier_industries_mask, model.feature_name_]
                
                # 进行预测
                predictions = model.predict(X_pred)
                
                # e) 【关键】将预测结果写回到我们的主DataFrame `full_df` 中
                #    这样，在预测下一天时，今天的预测值就可以作为昨天的“真实值”来计算滞后特征
                full_df.loc[
                    (full_df['date'] == current_date) & (full_df['行业名称'].isin(predict_day_data.loc[tier_industries_mask, '行业名称'])),
                    'load_MWh'
                ] = predictions

    # 6. 预测完成，筛选出未来的预测结果并返回
    final_predictions = full_df[full_df['date'] > last_known_date][['date', '行业名称', 'load_MWh']]
    final_predictions.rename(columns={'load_MWh': 'predicted_load_MWh'}, inplace=True)
    
    print("\n--- 预测完成！ ---")
    return final_predictions

In [ ]:
# --- 假设我们用全部历史数据来进行一次模拟预测 ---
# df 是我们最开始加载的、清洗好的完整历史数据集
# final_models 是我们调优后训练好的模型字典
# industry_avg_load 是包含了行业层级信息的DataFrame

if 'df' in locals() and 'final_models' in locals() and 'industry_avg_load' in locals():
    # 调用预测函数，预测未来8天的电量
    future_forecast = predict_future(
        historical_data=df, 
        days_to_predict=8, 
        models=final_models, 
        industry_tiers=industry_avg_load
    )

    print("\n未来8天电量预测结果预览：")
    print(future_forecast.head(10))

    # (可选) 查看某个您关心的行业的未来预测
    print("\n查看'14.化学原料和化学制品制造业'的未来8天预测：")
    print(future_forecast[future_forecast['行业名称'] == '14.化学原料和化学制品制造业'])

else:
    print("错误：缺少必要的输入数据（df, final_models, 或 industry_avg_load），无法进行预测。")